# `precip-dot` data test `04`:
## Sane values in final data

Verify that estimates in the final data are sane. This means no negative estimates or confidence bounds.

### inputs

The path to the directory containing the output directories for all steps of the pipeline needs to be saved in the `PIPE_DIR`.

In [1]:
def sanity_check(fp):
    """
    sanity check all data in a final output file
    """
    def sane_values(arr):
        arr = arr.flatten()
        return arr[~np.isnan(arr)] >= 0
    vars = ["pf", "pf_lower", "pf_upper"]
    with xr.open_dataset(fp) as ds:
        results = [np.all(sane_values(ds[var].values)) for var in vars]
    return (fp, results)
   
    
def run_test(data_dir):
    """
    run test on output data directory
    """
    print("Beginning test of consistent estimates in final data.\n")

    # check on all data files
    fps = glob.glob(os.path.join(data_dir, "*.nc"))

    p = Pool(20)
    results = p.map(sanity_check, fps)
    p.close()
    p.join()
    
    test_result = np.all([result[1] for result in results])
    # print results
    if test_result:
        print("\nTest result: PASS")
    else:
        print("\nTest result: FAIL\n")
    
    return results


In [2]:
import os, time, datetime, glob
import numpy as np
import xarray as xr
from multiprocessing import Pool

data_dir = os.path.join(os.getenv("PIPE_DIR"), "fudgeci")

tic = time.perf_counter()

_ = run_test(data_dir)

print("Elapsed time: {} m\n".format(round((time.perf_counter() - tic) / 60, 1)))

utc_time = datetime.datetime.utcnow()
print("Completion time of previous test: {}".format(utc_time.strftime("%Y-%m-%d %H:%M:%S")))

/home/UA/kmredilla/.localpython/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Beginning test of consistent estimates in final data.


Test result: PASS
Elapsed time: 3.3 m

Completion time of previous test: 2020-12-11 01:43:15
